In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [28]:
import json

with open('./data/zh_meta_part_1.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    print(f"result: {result}")
    print(isinstance(result, dict))
    break

result: {'content': '如果您的瀏覽器無安裝 Flash Player，請改用 Google Chrome、Firefox 或 Microsoft Edge 等瀏覽器，進入新版本包廂。如果您一定要使用 Flash Player，請看 Flash Player無法使用解決辦法教學。下載 Chrome 瀏覽器下載 Firefox 瀏覽器下載 Edge 瀏覽器', 'warc_headers': {'warc-type': 'conversion', 'warc-refers-to': '<urn:uuid:07418760-251b-4ce7-90c2-5338261d7287>', 'content-type': 'text/plain', 'warc-date': '2021-11-28T11:10:49Z', 'warc-target-uri': 'http://1232785.aiavtv7f.com/V4/?AID=151524&FID=1232785&WEBID=AVSHOW', 'content-length': '298', 'warc-identified-content-language': 'zho,eng', 'warc-record-id': '<urn:uuid:915d1507-d1e1-49f1-b086-d9d94f7050b1>', 'warc-block-digest': 'sha1:KCOWZRQDU5K6TUFRZB3GSJR4DG3QOUPG'}, 'metadata': {'identification': {'label': 'zh', 'prob': 0.9202119}, 'annotation': ['tiny'], 'sentence_identifications': [{'label': 'zh', 'prob': 0.9202119}]}}
True


In [34]:
#! pip install tokenizers

from pathlib import Path

from tokenizers import ByteLevelBPETokenizer


# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

contents = [json.loads(x)['content'] for x in json_list]
# Build an iterator over this dataset
def batch_iterator():
    batch_length = 1000
    for i in range(0, len(contents), batch_length):
        yield contents[i]


# And finally train
tokenizer.train_from_iterator(batch_iterator(), length=len(contents), vocab_size=52_000,min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Customize training
# tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])


# Save files to disk
tokenizer.save_model("./models", "cyborgoat-small")


['./model/cyborgoat-small-vocab.json', './model/cyborgoat-small-merges.txt']

In [35]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./models/cyborgoat-small-vocab.json",
    "./models/cyborgoat-small-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)


Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [41]:
output = tokenizer.encode("你好吗兄弟。")
output2 = tokenizer.encode("你好吗兄弟。")
output

Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [39]:
output.tokens

['<s>', 'ä½ł', 'å¥½', 'åĲĹ', 'åħĦ', 'å¼Ł', 'ãĢĤ', '</s>']

In [40]:
output.ids

[0, 996, 439, 6667, 16808, 6127, 409, 2]

In [42]:
output2.ids

[0, 996, 439, 6667, 16808, 6127, 409, 2]

In [43]:
from torch.utils.data import Dataset

class CorpustoDataset(Dataset):
    def __init__(self, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "./models/cyborgoat-small-vocab.json",
            "./models/cyborgoat-small-merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = contents[:1000] if evaluate else contents

        # src_files = Path("./data/").glob("*-eval.txt") if evaluate else Path("./data/").glob("*-train.txt")
        # for src_file in src_files:
        #     print("🔥", src_file)
        #     lines = src_file.read_text(encoding="utf-8").splitlines()
        #     self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])
